In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft as fft
import numpy as np
import pandas as pd
import re

In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
from datasets import load_dataset
datasets=load_dataset('wikitext','wikitext-2-raw-v1')

README.md: 0.00B [00:00, ?B/s]